In [1]:
  import sqlite3

  conn = sqlite3.connect('alleallu.db')
  cursor = conn.cursor()

  # Enable foreign key support
  cursor.execute("PRAGMA foreign_keys = ON;")

  # 1. Customers (Independent)
  cursor.execute('''
  CREATE TABLE IF NOT EXISTS Customers (
      customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
      customer_fname TEXT NOT NULL,
      customer_lname TEXT NOT NULL,
      phone_number TEXT UNIQUE NOT NULL,
      date_of_birth TEXT NOT NULL,
      customer_email TEXT UNIQUE NOT NULL,
      gender TEXT CHECK (gender IN('Male','Female','Unknown')) NOT NULL,
      street_address TEXT NOT NULL,
      city TEXT NOT NULL,
      postal_code TEXT NOT NULL,
      country TEXT NOT NULL
  );
  ''')

  # 2. Suppliers (Independent)
  cursor.execute('''
  CREATE TABLE IF NOT EXISTS Suppliers (
      supplier_id INTEGER PRIMARY KEY AUTOINCREMENT,
      supplier_name TEXT NOT NULL,
      phone_number TEXT UNIQUE NOT NULL,
      supplier_email TEXT UNIQUE NOT NULL,
      street_address TEXT NOT NULL,
      city TEXT NOT NULL,
      postal_code TEXT NOT NULL,
      country TEXT NOT NULL
  );
  ''')

  # 3. Products (Independent)
  cursor.execute('''
  CREATE TABLE IF NOT EXISTS Products (
      product_id INTEGER PRIMARY KEY AUTOINCREMENT,
      product_name TEXT NOT NULL,
      product_category TEXT NOT NULL,
      price REAL NOT NULL,
      cost REAL NOT NULL,
      brand TEXT NOT NULL
  );
  ''')

  # 4. Orders (Depends on Customers)
  cursor.execute('''
  CREATE TABLE IF NOT EXISTS Orders (
      order_id INTEGER PRIMARY KEY AUTOINCREMENT,
      order_date TEXT NOT NULL,
      order_status TEXT CHECK(order_status IN ('Pending','Shipped','Delivered','Cancelled')) NOT NULL,
      customer_id INTEGER,
      street_address TEXT NOT NULL,
      city TEXT NOT NULL,
      postal_code TEXT NOT NULL,
      country TEXT NOT NULL,
      FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
  );
  ''')

  # 5. Inventory (Depends on Products)
  cursor.execute('''
  CREATE TABLE IF NOT EXISTS Inventory (
      inventory_id INTEGER PRIMARY KEY AUTOINCREMENT,
      product_id INTEGER NOT NULL,
      quantity_in_stock INTEGER NOT NULL,
      location TEXT CHECK (location IN ('Warehouse 1','Warehouse 2','Warehouse 3','Warehouse 4','Warehouse 5')) NOT NULL,
      reorder_level INTEGER NOT NULL,
      batch_number TEXT NOT NULL,
      expiration_date TEXT NOT NULL,
      max_capacity INTEGER NOT NULL,
      status TEXT CHECK (status IN ('Available','Low Stock','Out of Stock')) NOT NULL,
      FOREIGN KEY (product_id) REFERENCES Products(product_id)
  );
  ''')

  # 6. Procurement (Depends on Products and Suppliers)
  cursor.execute('''
  CREATE TABLE IF NOT EXISTS Procurement (
      procurement_id INTEGER PRIMARY KEY AUTOINCREMENT,
      procurement_date TEXT NOT NULL,
      quantity INTEGER NOT NULL,
      total_cost REAL NOT NULL,
      unit_price REAL GENERATED ALWAYS AS (total_cost/quantity) STORED,
      delivery_date TEXT NOT NULL,
      payment_status TEXT CHECK (payment_status IN ('Pending','Paid','Overdue')) NOT NULL,
      supplier_id INTEGER NOT NULL,
      product_id INTEGER NOT NULL,
      FOREIGN KEY (product_id) REFERENCES Products(product_id),
      FOREIGN KEY (supplier_id) REFERENCES Suppliers(supplier_id)
  );
  ''')

  # 7. Shipments (Depends on Orders)
  cursor.execute('''
  CREATE TABLE IF NOT EXISTS Shipments (
      shipment_id INTEGER PRIMARY KEY AUTOINCREMENT,
      order_id INTEGER NOT NULL,
      estimated_delivery_date TEXT NOT NULL,
      delivery_date TEXT,
      delivery_status TEXT CHECK (delivery_status IN ('Pending','Delivered')) NOT NULL,
      shipping_status TEXT CHECK (shipping_status IN ('On Time','Delayed', NULL)),
      tracking_number TEXT UNIQUE NOT NULL,
      FOREIGN KEY (order_id) REFERENCES Orders(order_id)
  );
  ''')

  # 8. Payments (Depends on Orders)
  cursor.execute('''
  CREATE TABLE IF NOT EXISTS Payments (
      payment_id INTEGER PRIMARY KEY AUTOINCREMENT,
      order_id INTEGER NOT NULL,
      payment_amount REAL NOT NULL,
      payment_date TEXT NOT NULL,
      transaction_id TEXT UNIQUE NOT NULL,
      payment_installment_no INTEGER NOT NULL,
      payment_type TEXT CHECK (payment_type IN ('Credit Card','Debit Card','PayPal','Google Pay','Apple Pay')) NOT NULL,
      payment_deadline TEXT NOT NULL,
      payment_status TEXT CHECK (payment_status IN ('Pending','Paid','Refunded')) NOT NULL,
      street_address TEXT NOT NULL,
      city TEXT NOT NULL,
      postal_code TEXT NOT NULL,
      country TEXT NOT NULL,
      FOREIGN KEY (order_id) REFERENCES Orders(order_id)
  );
  ''')

  # 9. OrderDetails (Depends on Orders and Products)
  cursor.execute('''
  CREATE TABLE IF NOT EXISTS OrderDetails (
      order_id INTEGER NOT NULL,
      product_id INTEGER NOT NULL,
      quantity INTEGER NOT NULL,
      price REAL NOT NULL,
      cost REAL NOT NULL,
      PRIMARY KEY (order_id, product_id),
      FOREIGN KEY (order_id) REFERENCES Orders(order_id),
      FOREIGN KEY (product_id) REFERENCES Products(product_id)
  );
  ''')

  conn.commit()
  print("Database schema created successfully.")

Database schema created successfully.


In [2]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

for table_name in tables:
    print(f"Table: {table_name[0]}")
    cursor.execute(f"PRAGMA table_info({table_name[0]});")
    columns = cursor.fetchall()
    for col in columns:
        print(f"  Column: {col[1]}, Type: {col[2]}, NotNull: {col[3]}, DefaultVal: {col[4]}, PrimaryKey: {col[5]}")
    print("-" * 20)

Table: Customers
  Column: customer_id, Type: INTEGER, NotNull: 0, DefaultVal: None, PrimaryKey: 1
  Column: customer_fname, Type: TEXT, NotNull: 1, DefaultVal: None, PrimaryKey: 0
  Column: customer_lname, Type: TEXT, NotNull: 1, DefaultVal: None, PrimaryKey: 0
  Column: phone_number, Type: TEXT, NotNull: 1, DefaultVal: None, PrimaryKey: 0
  Column: date_of_birth, Type: TEXT, NotNull: 1, DefaultVal: None, PrimaryKey: 0
  Column: customer_email, Type: TEXT, NotNull: 1, DefaultVal: None, PrimaryKey: 0
  Column: gender, Type: TEXT, NotNull: 1, DefaultVal: None, PrimaryKey: 0
  Column: street_address, Type: TEXT, NotNull: 1, DefaultVal: None, PrimaryKey: 0
  Column: city, Type: TEXT, NotNull: 1, DefaultVal: None, PrimaryKey: 0
  Column: postal_code, Type: TEXT, NotNull: 1, DefaultVal: None, PrimaryKey: 0
  Column: country, Type: TEXT, NotNull: 1, DefaultVal: None, PrimaryKey: 0
--------------------
Table: sqlite_sequence
  Column: name, Type: , NotNull: 0, DefaultVal: None, PrimaryKey: 0


In [3]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving OrderDetails.csv to OrderDetails.csv
Saving Procurement.csv to Procurement.csv
Saving Products.csv to Products.csv
Saving Customer.csv to Customer.csv
Saving Inventory.csv to Inventory.csv
Saving Orders.csv to Orders.csv
Saving Payments.csv to Payments.csv
Saving Shipments.csv to Shipments.csv
Saving Supplier.csv to Supplier.csv
User uploaded file "OrderDetails.csv" with length 50899 bytes
User uploaded file "Procurement.csv" with length 39497 bytes
User uploaded file "Products.csv" with length 16088 bytes
User uploaded file "Customer.csv" with length 69338 bytes
User uploaded file "Inventory.csv" with length 14300 bytes
User uploaded file "Orders.csv" with length 58601 bytes
User uploaded file "Payments.csv" with length 95789 bytes
User uploaded file "Shipments.csv" with length 38832 bytes
User uploaded file "Supplier.csv" with length 3653 bytes


In [12]:
import csv
from datetime import datetime
import sqlite3

def standardize_date(date_str):
    """Convert a date string to ISO8601 (YYYY-MM-DD).
    Try several common formats."""
    for fmt in ('%Y-%m-%d', '%Y/%m/%d', '%d-%m-%Y', '%d/%m/%Y'):
        try:
            dt = datetime.strptime(date_str, fmt)
            return dt.strftime('%Y-%m-%d')
        except ValueError:
            continue
    # If no format matches, return the original string or raise an error
    return date_str

def import_csv_to_table(csv_file, table_name, cursor):
    # Map table names to list of column indices that are dates.
    # Adjust these indices based on your CSV file column order.
    date_fields = {
        'Customers': [4],  # date_of_birth is the 4th column (0-indexed)
        'Orders': [1],     # order_date is the 1st column
        'Procurement': [1, 4],  # procurement_date (col 1) and delivery_date (col 4)
        'Shipments': [1, 2],    # estimated_delivery_date (col 1) and delivery_date (col 2)
        'Payments': [3, 7]      # payment_date (col 3) and payment_deadline (col 7)
    }

    with open(csv_file, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        headers = next(csv_reader)  # Skip header row
        for row in csv_reader:
            # If there are designated date fields for this table, standardize them.
            if table_name in date_fields:
                for idx in date_fields[table_name]:
                    if idx < len(row) and row[idx]:
                        row[idx] = standardize_date(row[idx])
            # Prepare the placeholders for each value
            placeholders = ', '.join(['?' for _ in row])
            # Using explicit INSERT (we assume CSV includes exactly the columns needed)
            sql = f"INSERT INTO {table_name} VALUES ({placeholders})"
            cursor.execute(sql, row)

# Example usage:
conn = sqlite3.connect('alleallu.db')
cursor = conn.cursor()
cursor.execute("PRAGMA foreign_keys = ON;")

try:
    # Adjust the order based on dependencies:
    import_csv_to_table('Customer.csv', 'Customers', cursor)
    import_csv_to_table('Supplier.csv', 'Suppliers', cursor)
    import_csv_to_table('Products.csv', 'Products', cursor)
    import_csv_to_table('Orders.csv', 'Orders', cursor)
    import_csv_to_table('Inventory.csv', 'Inventory', cursor)
    import_csv_to_table('Procurement.csv', 'Procurement', cursor)
    import_csv_to_table('Shipments.csv', 'Shipments', cursor)
    import_csv_to_table('Payments.csv', 'Payments', cursor)
    import_csv_to_table('OrderDetails.csv', 'OrderDetails', cursor)
    conn.commit()
    print("Data imported successfully!")
except Exception as e:
    print(f"An error occurred: {e}")
    conn.rollback()

An error occurred: UNIQUE constraint failed: Customers.customer_id


In [5]:
import pandas as pd
# Query all three tables and load into pandas DataFrames
customers_df = pd.read_sql_query("SELECT * FROM Customers", conn)
orders_df = pd.read_sql_query("SELECT * FROM Orders", conn)
paymetns_df = pd.read_sql_query("SELECT * FROM Payments", conn)
shipments_df = pd.read_sql_query("SELECT * FROM Shipments", conn)

#add in the other tables


# Show the first 5 lines of each DataFrame
print("Customers Table:")
print(customers_df.head(5))
print("\nOrders Table:")
print(orders_df.head(5))
print("\nPayments Table:")
print(paymetns_df.head(5))
print("\nShipments Table:")
print(shipments_df.head(5))

Customers Table:
   customer_id customer_fname customer_lname      phone_number date_of_birth  \
0     10469737          James          Brown  +44 7495 939 652    1966-08-06   
1     10687271          David         Deleon  +44 7567 105 549    2004-03-20   
2     11034021           Jill         Harper  +44 7918 209 390    1988-10-30   
3     11241916           Dawn      Hernandez  +44 7921 979 419    2004-07-02   
4     11938386          Cindy           Ross  +44 7495 640 101    1996-03-13   

             customer_email   gender       street_address               city  \
0   james.brown@hotmail.com     Male  058 Rodriguez Ranch       East Natalie   
1  david.deleon@outlook.com     Male     1436 Horn Tunnel      Hendersonside   
2   jill.harper@outlook.com   Female    13266 Campos Fort  North Mariahshire   
3  dawn.hernandez@yahoo.com  Unknown      903 Danny Hills       Port Cynthia   
4      cindy.ross@yahoo.com  Unknown   1924 Michael Inlet      South Barbara   

  postal_code        

In [49]:
# Report 1: Sales Over Time
sales_ot_df = pd.read_sql_query('''SELECT
    strftime('%Y-%m', o.order_date) AS month,
    ROUND(SUM(od.price * od.quantity), 2) AS total_revenue,
    ROUND(SUM((od.price - od.cost) * od.quantity), 2) AS total_profit
FROM Orders o
JOIN OrderDetails od ON o.order_id = od.order_id
JOIN Payments p ON o.order_id = p.order_id
WHERE p.payment_status != 'Refunded' AND o.order_status != 'Cancelled'
GROUP BY month
ORDER BY month;
''', conn)

sales_ot_df.to_csv("sales_over_time_report.csv", index=False)
sales_ot_df.head(25)

month  total_revenue  total_profit
0   2023-01        4201.76       1788.12
1   2023-02        3653.48       1490.23
2   2023-03        4472.05       1840.05
3   2023-04        4416.78       1798.27
4   2023-05        4741.27       2020.90
5   2023-06        4197.18       1731.11
6   2023-07        5647.39       2371.26
7   2023-08        6544.59       2717.91
8   2023-09        4147.56       1716.07
9   2023-10        4165.41       1783.02
10  2023-11        4858.62       2110.39
11  2023-12        7587.61       3184.59
12  2024-01        4890.53       2020.25
13  2024-02        2134.35        882.51
14  2024-03        5085.02       2112.88
15  2024-04        3408.43       1430.43
16  2024-05        2917.27       1249.43
17  2024-06        4492.89       1903.14
18  2024-07        4026.42       1668.05
19  2024-08        3788.78       1539.02
20  2024-09        5943.38       2573.50
21  2024-10        3394.47       1380.32
22  2024-11        3707.11       1590.44
23  2024-12        3366.23       1405.29

In [50]:
# Report 2: Customer Segmentation by age & gender
segmetns_df = pd.read_sql_query("""
SELECT
    CASE
        WHEN (strftime('%Y', 'now') - strftime('%Y', date_of_birth)) BETWEEN 18 AND 25 THEN '18-25'
        WHEN (strftime('%Y', 'now') - strftime('%Y', date_of_birth)) BETWEEN 26 AND 35 THEN '26-35'
        WHEN (strftime('%Y', 'now') - strftime('%Y', date_of_birth)) BETWEEN 36 AND 50 THEN '36-50'
        WHEN (strftime('%Y', 'now') - strftime('%Y', date_of_birth)) BETWEEN 50 AND 64 THEN '50-64'
        ELSE '65+'
    END AS age_group,
    gender,
    COUNT(DISTINCT c.customer_id) AS total_customers,
    ROUND(SUM(od.price * od.quantity), 2) AS total_revenue,
    ROUND(SUM((od.price - od.cost) * od.quantity), 2) AS total_profit,
    ROUND(SUM(od.price * od.quantity) * 100.0 / SUM(SUM(od.price * od.quantity)) OVER (), 2) AS revenue_percentage,
    ROUND(SUM((od.price - od.cost) * od.quantity) * 100.0 / SUM(SUM((od.price - od.cost) * od.quantity)) OVER (), 2) AS profit_percentage
FROM Customers c
JOIN Orders o ON c.customer_id = o.customer_id
JOIN OrderDetails od ON o.order_id = od.order_id
JOIN Payments p ON o.order_id = p.order_id
WHERE p.payment_status != 'Refunded'
  AND o.order_status != 'Cancelled'
GROUP BY age_group, gender
ORDER BY total_revenue DESC;
""", conn)


segmetns_df.to_csv("customer_segment_revenue_report.csv", index=False)
segmetns_df.head(25)

age_group   gender  total_customers  total_revenue  total_profit  \
0      36-50   Female               49       17663.58       7499.69   
1      26-35  Unknown               47       16732.65       6953.81   
2      26-35   Female               38       14947.98       6214.69   
3      36-50  Unknown               40       13141.39       5457.43   
4      18-25   Female               32        9419.65       4029.73   
5      50-64   Female               20        8824.17       3576.92   
6      50-64  Unknown               23        8513.69       3553.99   
7      18-25  Unknown               15        6169.47       2626.14   
8      26-35     Male               13        4881.78       2028.02   
9      50-64     Male                4        2096.71        868.89   
10     36-50     Male                6        2039.65        920.71   
11     18-25     Male                4        1357.86        577.16   

    revenue_percentage  profit_percentage  
0                16.70              16.93  
1                15.82              15.69  
2                14.13              14.03  
3                12.42              12.32  
4                 8.90               9.09  
5                 8.34               8.07  
6                 8.05               8.02  
7                 5.83               5.93  
8                 4.61               4.58  
9                 1.98               1.96  
10                1.93               2.08  
11                1.28               1.30

In [2]:
# Report 3: YoY performance by Category
yoy_categ = pd.read_sql_query("""
SELECT
    pr.product_category,
    strftime('%Y', o.order_date) AS year,
    ROUND(SUM(od.price * od.quantity), 2) AS total_revenue,
    ROUND(SUM((od.price - od.cost) * od.quantity), 2) AS total_profit,
    ROUND(SUM((od.price - od.cost) * od.quantity) * 100.0 / SUM(od.price * od.quantity), 2) AS profit_margin,
    ROUND(SUM(od.price * od.quantity) * 100.0 / SUM(SUM(od.price * od.quantity)) OVER (PARTITION BY strftime('%Y', o.order_date)), 2) AS revenue_percentage
FROM Orders o
JOIN Payments p ON o.order_id = p.order_id
JOIN OrderDetails od ON o.order_id = od.order_id
JOIN Products pr ON od.product_id = pr.product_id
WHERE p.payment_status != 'Refunded' AND strftime('%Y', o.order_date) != '2025'
GROUP BY pr.product_category, strftime('%Y', o.order_date)
ORDER BY pr.product_category, total_revenue DESC;
""", conn)

yoy_categ.to_csv("category_yoy.csv", index=False)
yoy_categ.head(20)

NameError: name 'pd' is not defined

In [52]:
# Assuming yoy_categ is already a DataFrame from your SQL query
pivot_df = yoy_categ.pivot(index='product_category', columns='year',
                           values=['total_revenue', 'total_profit', 'profit_margin', 'revenue_percentage'])

# Flatten the MultiIndex columns:
pivot_df.columns = [f"{metric}_{year}" for metric, year in pivot_df.columns]
pivot_df = pivot_df.reset_index()

# Calculate changes between 2024 and 2023:
pivot_df['change in revenue'] = pivot_df['total_revenue_2024'] - pivot_df['total_revenue_2023']
pivot_df['change in profit'] = pivot_df['total_profit_2024'] - pivot_df['total_profit_2023']
pivot_df['change in profit margin'] = pivot_df['profit_margin_2024'] - pivot_df['profit_margin_2023']
pivot_df['change in revenue percentage'] = pivot_df['revenue_percentage_2024'] - pivot_df['revenue_percentage_2023']

pivot_df.to_csv("category_yoy_pivot.csv", index=False)
print(pivot_df)


  product_category  total_revenue_2023  total_revenue_2024  total_profit_2023  \
0            Blush             6052.75             4720.08            2381.76   
1          Bronzer             2215.52             1784.57             889.26   
2        Concealer             5046.42             2465.27            2047.74   
3         Eyeliner             8400.24             5913.67            3399.19   
4        Eyeshadow             7378.56             8167.21            3265.75   
5       Foundation             4861.10             3659.05            2011.75   
6      Highlighter             6463.47             6916.37            2792.68   
7         Lipstick             5801.46             5404.03            2430.80   
8          Mascara             5164.01             4788.06            2161.75   
9         Skincare             7250.17             3336.57            3171.24   

   total_profit_2024  profit_margin_2023  profit_margin_2024  \
0            1868.51               39.35    

In [53]:
#Report 4: On-time Delivery
delivery_perf = pd.read_sql_query("""
SELECT
    strftime('%Y-%m',delivery_date) AS shipment_month,
    SUM(CASE WHEN shipping_status = 'On Time' THEN 1 ELSE 0 END) AS on_time_shipments,
    SUM(CASE WHEN shipping_status = 'Delayed' THEN 1 ELSE 0 END) AS delayed_shipments,
    SUM(CASE WHEN shipping_status = 'Delayed' OR shipping_status = 'On Time' THEN 1 ELSE 0 END) AS total_shipments,
    ROUND(SUM(CASE WHEN shipping_status = 'On Time' THEN 1.0 ELSE 0 END) * 100.0 / SUM(CASE WHEN shipping_status = 'Delayed' OR shipping_status = 'On Time' THEN 1 ELSE 0 END), 2) AS on_time_delivery_rate
FROM Shipments
WHERE delivery_status = 'Delivered'
GROUP BY shipment_month
ORDER BY shipment_month;
""", conn)

delivery_perf.to_csv("delivery_perf.csv", index=False)

delivery_perf.head(20)

shipment_month  on_time_shipments  delayed_shipments  total_shipments  \
0         2023-01                  8                  2               10   
1         2023-02                 24                  1               25   
2         2023-03                 10                  1               11   
3         2023-04                 12                  2               14   
4         2023-05                 15                  4               19   
5         2023-06                 11                  2               13   
6         2023-07                 18                  3               21   
7         2023-08                 19                  3               22   
8         2023-09                 23                  1               24   
9         2023-10                 13                  0               13   
10        2023-11                 11                  2               13   
11        2023-12                 24                  4               28   
12        2024-01                 16                  1               17   
13        2024-02                  7                  1                8   
14        2024-03                 16                  1               17   
15        2024-04                 13                  1               14   
16        2024-05                 13                  2               15   
17        2024-06                 19                  1               20   
18        2024-07                 13                  1               14   
19        2024-08                 19                  3               22   

    on_time_delivery_rate  
0                   80.00  
1                   96.00  
2                   90.91  
3                   85.71  
4                   78.95  
5                   84.62  
6                   85.71  
7                   86.36  
8                   95.83  
9                  100.00  
10                  84.62  
11                  85.71  
12                  94.12  
13                  87.50  
14                  94.12  
15                  92.86  
16                  86.67  
17                  95.00  
18                  92.86  
19                  86.36